In [9]:
import os
import torch
import shutil

root = r"C:\Prgrm\ICLR26\features"
NEW_AVG = "mean_std"

def update_avg(header, obj, fpath):
    if isinstance(header, dict):
        if "avg" in header:
            old = header["avg"]
            print(f"     • Текущее header['avg'] = {old}")
            if old is True or old == "mean":
                print(f"     ✅ Обновляем: {old} → {NEW_AVG}")
                header["avg"] = NEW_AVG
                backup_path = fpath + ".bak"
                shutil.copy2(fpath, backup_path)
                print(f"     💾 Бэкап сохранён: {backup_path}")
                torch.save(obj, fpath)
            else:
                print("     ⏭ Не требует обновления")
        else:
            print("     ⚠ В dict header нет ключа 'avg'")
    else:
        if hasattr(header, "avg"):
            old = header.avg
            print(f"     • Текущее header.avg = {old}")
            if old is True or old == "mean":
                print(f"     ✅ Обновляем: {old} → {NEW_AVG}")
                header.avg = NEW_AVG
                backup_path = fpath + ".bak"
                shutil.copy2(fpath, backup_path)
                print(f"     💾 Бэкап сохранён: {backup_path}")
                torch.save(obj, fpath)
            else:
                print("     ⏭ Не требует обновления")
        else:
            print("     ⚠ header без поля avg")


print(f"=== Начинаем обход {root} ===")
for dirpath, _, files in os.walk(root):
    print(f"\n📂 Папка: {dirpath}")
    pt_files = [f for f in files if f.endswith(".pt")]
    if not pt_files:
        print("  ⏭ Нет .pt файлов")
        continue

    for fname in pt_files:
        fpath = os.path.join(dirpath, fname)

        if "avg-mean_std" not in fpath:
            print(f"  ⏭ Пропуск {fname} (нет avg-mean_std в пути)")
            continue

        print(f"  🔍 Обработка файла: {fname}")
        try:
            obj = torch.load(fpath, map_location="cpu")
            if isinstance(obj, dict) and "header" in obj:
                update_avg(obj["header"], obj, fpath)
            else:
                print("     ⚠ Нет ключа 'header' в obj")
        except Exception as e:
            print(f"     ❌ Ошибка загрузки: {e}")

print("\n=== Обход завершён ===")


=== Начинаем обход C:\Prgrm\ICLR26\features ===

📂 Папка: C:\Prgrm\ICLR26\features
  ⏭ Нет .pt файлов

📂 Папка: C:\Prgrm\ICLR26\features\bah
  ⏭ Нет .pt файлов

📂 Папка: C:\Prgrm\ICLR26\features\bah\dev
  ⏭ Нет .pt файлов

📂 Папка: C:\Prgrm\ICLR26\features\bah\dev\audio
  ⏭ Нет .pt файлов

📂 Папка: C:\Prgrm\ICLR26\features\bah\dev\audio\audio__clapa-laion-clap-htsat-fused__avg-mean_std__pv-v1
  🔍 Обработка файла: feats_seed42_subset0_avg-mean_std.pt
     • Текущее header['avg'] = mean
     ✅ Обновляем: mean → mean_std
     💾 Бэкап сохранён: C:\Prgrm\ICLR26\features\bah\dev\audio\audio__clapa-laion-clap-htsat-fused__avg-mean_std__pv-v1\feats_seed42_subset0_avg-mean_std.pt.bak
  🔍 Обработка файла: feats_seed42_subset5_avg-mean_std.pt
     • Текущее header['avg'] = mean
     ✅ Обновляем: mean → mean_std
     💾 Бэкап сохранён: C:\Prgrm\ICLR26\features\bah\dev\audio\audio__clapa-laion-clap-htsat-fused__avg-mean_std__pv-v1\feats_seed42_subset5_avg-mean_std.pt.bak

📂 Папка: C:\Prgrm\ICLR26\fe

In [6]:
import os
import torch

root = r"C:\Prgrm\ICLR26\features"

target_folders = ["avg-mean_std", "avg-mean"]   # какие подпапки обрабатывать
replace_values = {
    "avg-mean_std": "mean_std",
    "avg-mean": "mean"
}

for dirpath, _, files in os.walk(root):
    for fname in files:
        if not fname.endswith(".pt"):
            continue
        for marker, new_val in replace_values.items():
            if marker in dirpath:  # проверяем по названию папки
                fpath = os.path.join(dirpath, fname)
                print(f"🔍 Обработка {fpath}")
                try:
                    obj = torch.load(fpath, map_location="cpu")
                    if isinstance(obj, dict) and "header" in obj:
                        header = obj["header"]
                        if isinstance(header, dict):
                            if "avg" in header and header["avg"] != new_val:
                                print(f"   ✏ Исправляем avg: {header['avg']} → {new_val}")
                                header["avg"] = new_val
                                torch.save(obj, fpath)
                            else:
                                print("   ✅ Уже ок")
                        else:
                            print("   ⚠ header не dict")
                except Exception as e:
                    print(f"   ❌ Ошибка {e}")


🔍 Обработка C:\Prgrm\ICLR26\features\bah\dev\audio\audio__clapa-laion-clap-htsat-fused__avg-mean_std__pv-v1\feats_seed42_subset0_avg-mean_std.pt
   ✅ Уже ок
🔍 Обработка C:\Prgrm\ICLR26\features\bah\dev\audio\audio__clapa-laion-clap-htsat-fused__avg-mean_std__pv-v1\feats_seed42_subset0_avg-mean_std.pt
   ✏ Исправляем avg: mean_std → mean
🔍 Обработка C:\Prgrm\ICLR26\features\bah\dev\audio\audio__clapa-laion-clap-htsat-fused__avg-mean_std__pv-v1\feats_seed42_subset5_avg-mean_std.pt
   ✅ Уже ок
🔍 Обработка C:\Prgrm\ICLR26\features\bah\dev\audio\audio__clapa-laion-clap-htsat-fused__avg-mean_std__pv-v1\feats_seed42_subset5_avg-mean_std.pt
   ✏ Исправляем avg: mean_std → mean
🔍 Обработка C:\Prgrm\ICLR26\features\bah\dev\audio\audio__clapa-laion-clap-htsat-fused__avg-mean__pv-v1\feats_seed42_subset0_avg-mean.pt
   ✏ Исправляем avg: True → mean
🔍 Обработка C:\Prgrm\ICLR26\features\bah\dev\behavior\behavior__clipt-openai-clip-vit-base-patch32__col-text_llm__avg-mean_std__pv-v1\feats_seed42_subse

In [10]:
import glob, os

root = r"C:\Prgrm\ICLR26\features"   # корневая папка

count = 0
for bak in glob.glob(os.path.join(root, "**", "*.bak"), recursive=True):
    try:
        os.remove(bak)
        print(f"🗑 Удалён: {bak}")
        count += 1
    except Exception as e:
        print(f"⚠ Ошибка при удалении {bak}: {e}")

print(f"\n✅ Всего удалено файлов-бэкапов: {count}")


🗑 Удалён: C:\Prgrm\ICLR26\features\bah\dev\audio\audio__clapa-laion-clap-htsat-fused__avg-mean_std__pv-v1\feats_seed42_subset0_avg-mean_std.pt.bak
🗑 Удалён: C:\Prgrm\ICLR26\features\bah\dev\audio\audio__clapa-laion-clap-htsat-fused__avg-mean_std__pv-v1\feats_seed42_subset5_avg-mean_std.pt.bak
🗑 Удалён: C:\Prgrm\ICLR26\features\bah\dev\behavior\behavior__clipt-openai-clip-vit-base-patch32__col-text_llm__avg-mean_std__pv-v1\feats_seed42_subset0_avg-mean_std.pt.bak
🗑 Удалён: C:\Prgrm\ICLR26\features\bah\dev\behavior\behavior__clipt-openai-clip-vit-base-patch32__col-text_llm__avg-mean_std__pv-v1\feats_seed42_subset5_avg-mean_std.pt.bak
🗑 Удалён: C:\Prgrm\ICLR26\features\bah\dev\face\face__clipv-openai-clip-vit-base-patch32__frames30__img224__avg-mean_std__pv-v1\feats_seed42_subset0_avg-mean_std.pt.bak
🗑 Удалён: C:\Prgrm\ICLR26\features\bah\dev\face\face__clipv-openai-clip-vit-base-patch32__frames30__img224__avg-mean_std__pv-v1\feats_seed42_subset5_avg-mean_std.pt.bak
🗑 Удалён: C:\Prgrm\ICL